# Download raw OSM files
The online query API only works for small areas, so we instead download OSM registries when processing Mapillary data.

1. download an `.osm.pbf` file
2. crop it to our region of interest
3. convert it to an XML `.osm` file, readable in Python.

In [17]:
!du -hs /data/users/psarlin/data/osm/*

In [18]:
%%bash

outdir="/data/users/psarlin/data/osm"

#prefix="north-america/us/california"
#infile="norcal-latest.osm"
#outfile="sanfrancisco.osm"
#longlat="-122.518043,37.740938,-122.375488,37.815516"

#prefix="europe/netherlands"
#infile="noord-holland-latest.osm"
#outfile="amsterdam.osm"
#longlat="4.815066,52.328212,4.95753,52.401213"

#prefix="europe/france"
#infile="pays-de-la-loire-latest.osm"
#outfile="lemans.osm"
#longlat="0.168179,47.987369,0.238288,48.027479"

#prefix="europe/germany"
#infile="berlin-latest.osm"
#outfile="berlin.osm"
#longlat="13.341396,52.453993,13.493914,52.555997"

#prefix="europe"
#infile="france-latest.osm"
#outfile="paris.osm"
#longlat="2.224122,48.802912,2.46976,48.902156"

#prefix="europe"
#infile="france-latest.osm"
#outfile="nantes.osm"
#longlat="-1.620973,47.177927,-1.486904,47.257056"

#prefix="europe"
#infile="france-latest.osm"
#outfile="toulouse.osm"
#longlat="1.419789,43.586468,1.459844,43.617663"

#prefix="europe"
#infile="lithuania-latest.osm"
#outfile="vilnius.osm"
#longlat="25.224609,54.658168,25.324724,54.717325"

prefix="north-america/us"
infile="new-york-latest.osm"
outfile="newyork.osm"
longlat="-74.101653,40.666145,-73.895545,40.857878"

prefix="north-america/us"
infile="new-jersey-latest.osm"
outfile="hoboken.osm"
longlat="-74.0564213504,40.7279393562,-74.0108754797,40.7725784278"

#prefix="europe"
#infile="finland-latest.osm"
#outfile="helsinki.osm"
#longlat="24.8495743636,60.1287961209,25.050229519,60.2100827695"

#prefix="europe"
#infile="italy-latest.osm"
#outfile="milan.osm"
#longlat="9.127869378,45.4412806912,9.241868129,45.5439770045"

#prefix="europe"
#infile="france-latest.osm"
#outfile="avignon.osm"
#longlat="4.7663933688,43.9232525928,4.8512743924,43.9737017839"

cd ~/fbsource/fbcode/spatial_computing/osmium-tool-build
https_proxy=fwdproxy:8080 wget https://github.com/osmcode/osmium-tool/archive/v1.10.0.tar.gz
tar xz --strip-components=1 -f v1.10.0.tar.gz

url="https://download.geofabrik.de/$prefix/$infile.pbf"
raw="$outdir/$infile.pbf"
if ! [ -f $raw ]; then
    curl $(fwdproxy-config curl) $url --output $raw
fi
buck run @mode/opt -c python.package_style=inplace //spatial_computing/osmium-tool-build:osmium extract -- -s smart -b $longlat $raw -o $outdir/$outfile.pbf --overwrite
buck run @mode/opt -c python.package_style=inplace //spatial_computing/osmium-tool-build:osmium cat -- $outdir/$outfile.pbf -o $outdir/$outfile --overwrite

hg purge .
cd -